# Spatial Joins

The notebooks on [Spatial Relations](https://github.com/jamesdamillington/john-snow/blob/main/code/python/Relations.ipynb), [Spatial Operations](https://github.com/jamesdamillington/john-snow/blob/main/code/python/Operations.ipynb) and [Voronoi](https://github.com/jamesdamillington/john-snow/blob/main/code/python/Voronoi.ipynb) showed how we can use the [Shapely](https://shapely.readthedocs.io/) and [PySAL](https://pysal.org/libpysal/) packages to analyse and create geometric features in space. But what about if we want to also analyse data associated with those geometric features?

For example, the `Cholera_Deaths.shp` shapefile [contains](https://github.com/jamesdamillington/john-snow/blob/main/README.md) the number of deaths due to Cholera reported at locations around Soho during the [1854 cholera outbreak](https://en.wikipedia.org/wiki/1854_Broad_Street_cholera_outbreak). How can we compare the number of deaths reported by this datafile to the buffers and Thiessen polygons (voronoi) created in those other notebooks?

To do this we need to do a [spatial join](http://wiki.gis.com/wiki/index.php/Spatial_Join) of the deaths point data and the buffer/voronoi polygon data. A spatial join allows us to get attribute data from one spatial layer and transfer it to another layer based on their spatial relationship (in contrast to Pandas non-spatial joins which [merge](https://pandas.pydata.org/docs/user_guide/merging.html) DataFrames without use of spatial geometries). 

We can do spatial joins using [the `sjoin` method in GeoPandas](https://geopandas.org/mergingdata.html#spatial-joins) (which in turn draws on [Shapely](https://shapely.readthedocs.io/) and [PySAL](https://pysal.org/libpysal/)). In this notebook we will:
- use `sjoin` to find points within <a href="#buff">buffer polygons</a>
    - review <a href="#buff">how joins and spatial relations align</a>
    - <a href="#compare">compare types of Join</a>
    - think about what <a href="#faulty">faulty spatial relations</a> might be
    - examine the implications of <a href="#faulty">overlaps between buffers</a>
- use `sjoin` to find points within <a href="#voronoi">voronoi polygons</a>
- <a href="#comp">compare the results</a> of joins for the buffers vs voronoi using a-spatial (bar) plots 

## Essentials of Spatial Joins  <a id='notes'></a>
Spatial Joins rely on concepts from both [Spatial relations](https://en.wikipedia.org/wiki/Spatial_relation) and [non-spatial joining of DataFrames](https://pandas.pydata.org/docs/user_guide/merging.html).   

When doing a spatial join, we need to decide two things:

- what spatial relation are we going to use to do the join? (defined by the `op` argument)
- which type of join between the two DataFrames? (defined by the `how` argument)

Possible spatial relations we can use [with `sjoin` in GeoPandas are](https://geopandas.org/mergingdata.html#spatial-joins):

- `intersects`: The attributes will be joined if the boundary and interior of the object intersect in any way with the boundary and/or interior of the other object.
- `within`: The attributes will be joined if the object’s boundary and interior intersect only with the interior of the other object (not its boundary or exterior).
- `contains`: The attributes will be joined if the object’s interior contains the boundary and interior of the other object and their boundaries do not touch at all.

These relations are identical to those with the same [Shapely function](https://shapely.readthedocs.io/en/stable/manual.html#binary-predicates) name (indeed, GeoPandas uses those exact functions itself). See the [Spatial Relations notebook](https://github.com/jamesdamillington/john-snow/blob/main/code/python/Relations.ipynb) for more on these. 

Possible types of join [are](https://geopandas.org/mergingdata.html#spatial-joins):

- `left`: use the index from the first (or left_df) geodataframe that you provide to sjoin; retain only the left_df geometry column
- `right`: use index from second (or right_df); retain only the right_df geometry column
- `inner`: use intersection of index values from both geodataframes; retain only the left_df geometry column

These determine which geometry is retained (i.e. from which DataFrame), and are the same as those used in [Pandas merge methods](https://pandas.pydata.org/docs/user_guide/merging.html#brief-primer-on-merge-methods-relational-algebra). 

## Setup

Import required libraries

In [ ]:
import geopandas as gpd
import shapely
print(shapely.__version__)

import matplotlib.pyplot as plt
import matplotlib as mpl   #for colour palette
  
from libpysal.cg import voronoi, voronoi_frames
import numpy as np
import pandas as pd 

Load data

In [ ]:
pumps = gpd.read_file('../../data/csds/snow7/pumps.shp')      #point locations of pumps
blocks = gpd.read_file('../../data/dani/polys.shp')           #polygons delineating blocks of buildings
deaths = gpd.read_file('../../data/dani/Cholera_Deaths.shp')  #locations with at least one death (incl death count)

The deaths data provides point locations with at least one death, with a count of the deaths at that location. Let's check the data:

In [ ]:
deaths.head()

The `Id` column currently just contains `0`s. Let's give each point a unique integer value:

In [ ]:
deaths['Id'] = [i for i in range(len(deaths['Id']))]
deaths.head()

In [ ]:
deaths.crs

The deaths dataset has the same CRS as the other data so we can readily use these datasets together, including for plotting:

In [ ]:
f, ax = plt.subplots(1, figsize=(9, 9))
blocks.plot(ax=ax, facecolor='0.9', linewidth=0)
pumps.plot(ax=ax, color='black')
deaths.plot(ax=ax, color='red', markersize=2)

# Points in Buffer Polygons  <a id='buff'></a>

In other notebooks we have seen how to create buffers around the Soho pumps. Let's create these again here so that we can use them to examine how deaths were related to pumps:

In [ ]:
pumps['buffer']=pumps.buffer(200)

In [ ]:
f, ax = plt.subplots(1, figsize=(9, 9))
blocks.plot(ax=ax, facecolor='0.9', linewidth=0)
pumps['geometry'].plot(ax=ax, color='black')
deaths.plot(ax=ax, color='red', markersize=2)
pumps['buffer'].plot(ax=ax, linewidth=2, edgecolor='#e66101', facecolor='#e66101', alpha=0.4)

The code above creating the buffer polygons added those polygons to the `pumps` DataFrame. This means we now have _two_ columns containing geometries in the `pumps` DF:

In [ ]:
pumps.head()

Having two 'geometry' columns will confuse things when doing a spatial join. So, first we will create a new DF dropping the pumps point geometries to leave only the the buffers polygons geometries, plus all the other data columns:

In [ ]:
buffers = pumps.loc[:, pumps.columns != 'geometry']
buffers

To do the spatial join, we use the [`sjoin` method](https://geopandas.org/reference/geopandas.sjoin.html) from GeoPandas. The `sjoin` method expects the DataFrames to each have a _geometry_ column. The `buffers` DF we just created no longer has a _geometry_ column, so we need to rename:

In [ ]:
buffers=buffers.rename(columns = {'buffer':'geometry'})
buffers

Now we are ready to do our spatial join of the `deaths` and `buffers` dataframes. We want to know which deaths occured within which buffers. So, referring to <a href="#notes">the notes above</a>, we can use the following code:

In [ ]:
deaths_within_buffers = gpd.sjoin(deaths, buffers, op='within', how='inner')  #use inner returns left (point) geometry (from deaths)
deaths_within_buffers

Note that we use the `within` relation, specifying `deaths` before `buffers` (as we want 'deaths within buffers'). We use `inner` which retains the left GDF geometry, which in this case is from the `deaths` GDF.

## Aligning Joins and Spatial Relations <a id='align'></a>

What if we had tried the `contains` spatial relation, specifying `buffers` before `deaths`. We might expect that to have similar effect as the spatial relation is essentially the same (*contains* being the reciprocal of *within*):

In [ ]:
buffers_contains_deaths = gpd.sjoin(buffers, deaths, op='contains', how='inner')  #inner returns left (polygon) geometry (from buffers)
buffers_contains_deaths

What's happened here? We have the same number of rows and columns returned, but now the geometry columns contains polygons not points. This is because the `buffers` GDF was specified in the 'left' position (i.e. it was the first GDF specified in the `sjoin` function) and we used `inner` which returns the geometry from the 'left' position. 

So could we just use `right`? That would return the geometry from the 'right' GDF:

In [ ]:
buffers_contains_deaths = gpd.sjoin(buffers, deaths, op='contains', how='right')  #right returns right (point) geometry (from deaths)
buffers_contains_deaths

Okay, so this looks better as we now have the point geometry.

But wait... look how many rows we now have: 

In [ ]:
print(buffers_contains_deaths.shape)
print(deaths_within_buffers.shape)

We see that `buffers_contains_deaths` has more points than `deaths_within_buffers`. This is because while `inner` returns only objects that match the spatial relation in both GDFs, `right` returns all objects from the 'right' GDF, regardless of whether they meet the requirements of the spatial relations. 

Let's see this:

In [ ]:
f, axes = plt.subplots(1,2, figsize=(12, 12))

for ax in axes:
    blocks.plot(ax=ax, facecolor='0.9', linewidth=0)
    pumps.plot(ax=ax, color='black')
    
    #plot the deaths points, small in blue (these will be over-laid)
    deaths.plot(ax=ax, color='blue', markersize=1)
    pumps['buffer'].plot(ax=ax, linewidth=2, edgecolor='#e66101', facecolor='#e66101', alpha=0.1)

#plot the sjoin points, larger in red
buffers_contains_deaths.plot(ax=axes[0], color='red', markersize=3)
deaths_within_buffers.plot(ax=axes[1], color='red', markersize=3)

#add titles
axes[0].set_title('buffers_contains_deaths (with "right")')
axes[1].set_title('deaths_within_buffer (with "inner")')
plt.show()

Hopefully you can see in the `deaths_within_buffer` plot that the points in the north east that are not within any buffers are a different colour... this is because they have not been overlaid by the points in the GDF created by `sjoin`, illustrating the difference in the two GDFs and how the combination of spatial relations and type of join can have varying effects.

## Comparing types of Join <a id='compare'></a>

Let's compare the three types of join for "deaths within buffers":

In [ ]:
dwb_inner = gpd.sjoin(deaths, buffers, op='within', how='inner')  
dwb_left = gpd.sjoin(deaths, buffers, op='within', how='left')  
dwb_right = gpd.sjoin(deaths, buffers, op='within', how='right')  

In [ ]:
print("inner shape: " + str(dwb_inner.shape))
print("left shape: " + str(dwb_left.shape))
print("right shape: " + str(dwb_right.shape))

`inner` and `right` have the same number of rows, but those rows contain different geometries (point vs polygon). And `left` returns all points, regardless of the spatial relation.   

In [ ]:
f, axes = plt.subplots(1,3, figsize=(12, 15))

dwb_inner.plot(ax=axes[0])
dwb_left.plot(ax=axes[1])
dwb_right.plot(ax=axes[2])

## Faulty Spatial Relations <a id='faulty'></a>

A quick reminder to make sure you get the order of the GDFs in `sjoin` correct, so that the spatial relation makes sense. For example, does asking for deaths points that *contain*  buffer polygons make sense? 

In [ ]:
deaths_contain_buffers = gpd.sjoin(deaths, buffers, op='contains', how='inner') 
deaths_contain_buffers

Unsurprisingly, we got no data points returned!

In [ ]:
deaths_contain_buffers.shape

## Overlaps <a id='overlaps'></a>

Before we move on to Thiessen Polygons, let's address the issue of overlaps in between our buffers. Above we have been creating GDFs containing ~500 rows. But... how many points are in the original deaths data?

In [ ]:
deaths.shape

So here we see that there are many more points (rows) in the `deaths_within_buffers` DF (for example) than in the `deaths` DF, even though we are not including those points not in any buffer.

This is because a death point can be within _multiple_ buffer polygons, due to overlaps (as we can see in the visualisation above). For example, remember how we gave each points in the `deaths` DF a unique `Id` right after we loaded it? Now we can use that to look for duplicates:

In [ ]:
deaths_within_buffers.sort_values('Id').tail()

See how the point with `Id == 249` is within the buffer of three different pumps?

If we want each point to be in a single polygon we will need to use Thiessen Polygons (which we'll look at below and compare). 

But we can use the info in the joined table to split out points by the buffer they are in. For example, we could plot points within two different buffers (in a different colour from `deaths` points): 

In [ ]:
f, axes = plt.subplots(1,2, figsize=(12, 12))

for ax in axes:
    blocks.plot(ax=ax, facecolor='0.9', linewidth=0)
    pumps.plot(ax=ax, color='black')
    
    #plot the deaths points, small in blue (these will be over-laid)
    deaths.plot(ax=ax, color='blue', markersize=1)

#plot the sjoin points, larger in red
deaths_within_buffers[deaths_within_buffers['name']=='Rupert St Pump'].plot(ax=axes[0], color='red', markersize=3)
deaths_within_buffers[deaths_within_buffers['name']=='Brewer St Pump'].plot(ax=axes[1], color='red', markersize=3)

#plot the individual buffers
buffers[buffers['name']=='Rupert St Pump'].plot(ax=axes[0], linewidth=2, edgecolor='#e66101', facecolor='#e66101', alpha=0.1)
buffers[buffers['name']=='Brewer St Pump'].plot(ax=axes[1], linewidth=2, edgecolor='#e66101', facecolor='#e66101', alpha=0.1)

#add titles
axes[0].set_title('Rupert St Buffer')
axes[1].set_title('Brewer St Buffer')
plt.show()

Because we are working with a DataFrame, our analysis is now not restricted only to spatial visualisation. For example, let's count the number of deaths points in each buffer using the [`value_counts` Pandas method](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.Series.value_counts.html): 

In [ ]:
deaths_within_buffers["name"].value_counts()

And we could plot a bar chart to compare the number of points between pump buffers:

In [ ]:
deaths_within_buffers["name"].value_counts().plot.bar(title="count of death points")
plt.show()

# Points in Voronoi  <a id='voronoi'></a>

Above we saw how a single death point could be located within multiple buffer polygons. To avoid this we could use Thiessen Polygons in a Voronoi diagram to ensure that points are located in only a single polygon (as Thiessen Polygons will never overlap). 

First we need to create our Thiessen Polygons (see the [Voronoi notebook](https://github.com/jamesdamillington/john-snow/blob/main/code/python/Voronoi.ipynb) for more on this):

In [ ]:
points = np.vstack([pumps['x'], pumps['y']]).T  
regions_df, vertices_df = voronoi_frames(points, clip='none')

As we have created DataFrames (using the `_frames` function) we can add some useful information to the voronoi geometries (useful for labelling plots later):

In [ ]:
regions_df['name'] = pumps['name']
vertices_df['name'] = pumps['name']
regions_df

Quick spatial visualisation to check this has worked: 

In [ ]:
f, ax = plt.subplots(1, figsize=(9, 9))
ax.set_xlim(529100,529700)   #play with these values
ax.set_ylim(180700,181400)   #play with these values
blocks.plot(ax=ax, facecolor='0.9', linewidth=0)
deaths.plot(ax=ax, facecolor='black', markersize=2)
#use column here with a colourmap 
regions_df.plot(ax=ax, column='name', cmap='Set1', edgecolor='black', alpha=0.3)
vertices_df.plot(ax=ax, color='red')

It looks like are we now ready to do our spatial join. So, referring to <a href="#notes">the notes above</a>, we can use the following code:

In [ ]:
deaths_within_voronoi = gpd.sjoin(deaths, regions_df, op='within', how='inner')
deaths_within_voronoi

We have successfully created a DataFrame, but note that we got a warning about the CRS of the 'right' DataFrame (`regions_df`). The warning implies that the DataFrame has no CRS. Let's check that:

In [ ]:
regions_df.crs

We got no output. So let's force something to be printed:

In [ ]:
print(regions_df.crs)

Now we can see no CRS is set for `regions_df`. Given that the last spatial visualisation looked good and the `deaths_in_voronoi` DF looks okay from a quick look, let's be safe and set the CRS to match the `deaths` DF CRS. 

Let's remind ourselves what that CRS is: 

In [ ]:
deaths.crs

And now set the CRS of `regions_df` using the epsg id:

In [ ]:
regions_df= regions_df.set_crs(epsg=27700)

And check:

In [ ]:
print(regions_df.crs)

Good, now do the spatial join again:

In [ ]:
deaths_within_voronoi = gpd.sjoin(deaths, regions_df, op='within', how='inner')
deaths_within_voronoi

Remember that for deaths in buffers the result of our `sjoin` was a DF with many more rows (points) than the original `deaths` DF. We can see here that we now have the same number of points (as hoped, given that Thiessen Polygons do not overlap):

In [ ]:
print(deaths.shape)
print(deaths_within_voronoi.shape)

As above, let's plot for a single polygon and points within to visualise the new DF:

In [ ]:
f, ax = plt.subplots(1, figsize=(9, 9))
blocks.plot(ax=ax, facecolor='0.9', linewidth=0)
pumps.plot(ax=ax, color='black')

#plot all points (these will be over-laid)
deaths.plot(ax=ax, color='blue', markersize=1)

#subset DFs by name to plot only one polygon and points within
regions_df[regions_df['name']=='Broad St Pump'].plot(ax=ax, linewidth=1, 
                                               edgecolor='#e66101', facecolor='none', alpha=0.6)
deaths_within_voronoi[deaths_within_voronoi['name']=='Broad St Pump'].plot(ax=ax, color='red', markersize=3)



ax.set_xlim(529100,529700)   #play with these values
ax.set_ylim(180700,181400)   #play with these values

plt.show()

Because each point is now unique to each pump polygon, we can visualise all points coloured according to their _nearest_ pump:

In [ ]:
pumpNames = list(deaths_within_voronoi['name'].unique())

f, ax = plt.subplots(1, figsize=(9, 9))
blocks.plot(ax=ax, facecolor='0.9', linewidth=0)
pumps.plot(ax=ax, color='black')

regions_df.plot(ax=ax, linewidth=1, edgecolor='#e66101', facecolor='none', alpha=0.4)

#loop through pump names, plotting points with a different colour
for idx, name in enumerate(pumpNames):
    deaths_within_voronoi[deaths_within_voronoi['name']==name].plot(ax=ax,color=mpl.cm.tab10(idx), markersize=10)

ax.set_xlim(529100,529700)   #play with these values
ax.set_ylim(180700,181400)   #play with these values

plt.show()

And as above for buffers, because we are working with a DataFrame, our analysis is now not restricted only to spatial visualisation. For example, let's count the number of deaths point locations in each buffer using the [`value_counts` Pandas method](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.Series.value_counts.html): 

In [ ]:
deaths_within_voronoi["name"].value_counts()

And we could plot a bar chart to compare the number of point locations between pump buffers:

In [ ]:
deaths_within_voronoi["name"].value_counts().plot.bar(title="count of death point locations")
plt.show()

# Comparing Points in Buffers vs Voronoi <a id='comp'></a>

We noted above that `sjoin` for buffers produced different output than for voronoi. Let's compare the differences. As we have joined the data in DataFrames we can use Pandas data manipulation and plotting methods. 

First, calculate DFs containing counts of the point locations in the polygons: 

In [ ]:
vcounts = deaths_within_voronoi["name"].value_counts()
bcounts = deaths_within_buffers["name"].value_counts()

Now combine the DataFrames we've created using a **non-spatial** join (i.e. Pandas `merge`):

In [ ]:
compareCounts = pd.merge(bcounts, vcounts, right_index = True,left_index = True)
compareCounts

The DF that we have produced, does not have intutive column names. So let's rename:

In [ ]:
compareCounts = compareCounts.rename(columns={
    'name_x':'buffers',
    'name_y':'voronoi'})
compareCounts

Now let's do a paired bar plot to compare the number of deaths points attributed to each pump by the two methods:

In [ ]:
compareCounts.sort_values('buffers',ascending=False).plot.bar(title="count of deaths point locations")
plt.show()

So we can see the buffers polygons always have more deaths points within them than the voronoi polygons. This makes sense given that all pump buffers overlap at least one other buffer. 

This analysis has just been for the _number of point locations_ in the `deaths` DF. But the Cholera Deaths point data [are](https://bitbucket.org/darribas/reproducible_john_snow/src/master/data/README.txt):

>Points for each location of one or more deaths. Attribute value gives number of deaths at that location

This means the `deaths` DF also contains information about the _total number of people_ that died at each location. So we can also compare the total number of deaths attributed to each polygon by the points in the two different approaches. We do this by calculating the **sum** of the _Count_ columns (rather the count, as above): 

In [ ]:
vdeaths = deaths_within_voronoi.groupby('name')['Count'].sum()
bdeaths = deaths_within_buffers.groupby('name')['Count'].sum()

Now create a single DF: 

In [ ]:
compareSums = pd.merge(bdeaths,vdeaths,right_index = True,left_index = True)
compareSums = compareSums.rename(columns={
    'Count_x':'buffers',
    'Count_y':'voronoi'})
compareSums

And a bar plot now compares total deaths attributed to pumps between the two approaches: 

In [ ]:
compareSums.sort_values('buffers',ascending=False).plot.bar(title="sum of death counts")
plt.show()

As for the number of point locations, we always have more deaths in buffers because some points are double-counted due to overlaps. 

We could check what the total number of death is in the original data:

In [ ]:
deaths['Count'].sum()

And then see the sum for the points in polygons for the two different approaches:

In [ ]:
compareSums.sum()

We can see that Voronoi, because of no spatial overlap, ensures no double-counting of deaths.

# Further Activities

- Explore spatial joins for other geometries, for example:
    - polygons within polygons
    - lines intersects polygons 
- Try [other tutorials](https://kodu.ut.ee/~kmoch/geopython2018/lessons/L3/spatial-join.html) on spatial joins 

# Credits!

## Contributors:
The following individual(s) have contributed to these teaching materials: James Millington (james.millington@kcl.ac.uk).

## License
These teaching materials are licensed under a mix of [The MIT License](https://opensource.org/licenses/mit-license.php) and the [Creative Commons Attribution-NonCommercial-ShareAlike 4.0 license](https://creativecommons.org/licenses/by-nc-sa/4.0/).